# Load data into the local vector database

This notebook requires that you have downloaded a set of plaintext or markdown documents into the `corpus` folder.

In my case I used the CC0 licensed repository https://github.com/ddmee/emanote/tree/master/content

To download a copy run the following commands:

```bash
git clone https://github.com/ddmee/emanote noterepo
mkdir corpus
cp noterepo/content/2022*.md corpus
rm -rf noterepo
```

In [1]:
import chromadb
import os

In [2]:
client = chromadb.PersistentClient(path="db/")
collection_name = "Corpus"
merged_model_dir = "merged-fine-tuned"
device = "cuda"
corpus_dir = "corpus"

In [3]:
if len(client.list_collections()) > 0:
    print("Removing collection")
    client.delete_collection(name=collection_name)

Removing collection


In [4]:
collection = client.create_collection(name=collection_name)

In [5]:
def add_file_to_collection(full_path, file_id):
    with open(full_path, 'rt') as f:
        doc = f.read()
        collection.add(documents=[doc], metadatas=[{"source": full_path}], ids=[f"{file_id:09}"])

In [6]:
file_id = 1
for file_name in os.listdir(corpus_dir):
    full_path = os.path.join(corpus_dir, file_name)
    if os.path.isfile(full_path):
        add_file_to_collection(full_path, file_id)
        file_id += 1

In [ ]:
collection.query(
    query_texts=["How do I minimize training error?"], 
    n_results=2
)